In [1]:
import openai
import os
import pickle

In [1]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT, QA_PROMPT

In [2]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], output_parser=None, partial_variables={}, template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:', template_format='f-string', validate_template=True)

In [3]:
QA_PROMPT

PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:", template_format='f-string', validate_template=True)

## Set API Keys

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## Build the Chatbot

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# The vectorstore we'll be using
from langchain.vectorstores import FAISS, Chroma

# The LangChain component we'll use to get the documents
from langchain.chains import RetrievalQA, ConversationalRetrievalChain, ChatVectorDBChain

# The easy document loader for text
from langchain.document_loaders import TextLoader

# The embedding engine that will convert our text to vectors
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.prompts.prompt import PromptTemplate

# Chat specific components
from langchain.memory import ConversationBufferMemory

from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [23]:
# Build the model
model_version = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.3, model_name=model_version)

# Load data
loader = UnstructuredPDFLoader("../documents/wftda_rules.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Load our data into embeddings
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(texts, embeddings)#, persist_directory=".")
# db.persist()

# Expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


In [25]:
with open("vectors_roller_derby.pkl", "wb") as f:
    pickle.dump(db, f)

In [5]:
# Setup our template to prompt the model to be a roller derby referee
start_template = """
You are an expert referee for the sport Roller Derby. Your goal is to help the user understand
the rules of Roller Derby, as well as assist in adjuticating any disputes confusing situations
that might occur where the rules are unclear or tricky to apply. You are friendly and helpful,
but also kind of sassy, because this is Roller Derby after all.
You are given the following extracted parts of the official roller derby rulebook. Please
provide a conversational answer.
Please give references to the rulebook when possible.
If you don't know the answer to a question, just say "Hmm, I'm not sure.".
Don't try to make up an answer.

Question: {question}
==========
{context}
==========
Autoref:"""
QA = PromptTemplate(template=start_template, input_variables=["question", "context"])

# Setup our intermediate template that runs inbetween questions.
intermediate_template = """
Given the following conversation and a follow up question, rephrease the following
question to be a standalone question.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(intermediate_template)

In [6]:
# Setup the memory system for the chatbot
memory = ConversationBufferMemory(memory_key="chat_history", ai_prefix="Autoref")

In [27]:
# Create a chain to answer questions
# qa = ConversationalRetrievalChain.from_llm(llm, 
#                                            retriever,
#                                            memory=memory,
#                                            qa_prompt=QA,
#                                            CONDENSE_QUESTION_PROMPT=CONDENSE_QUESTION_PROMPT)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    # verbose=True,
    # qa_prompt=QA,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
)

In [19]:
# chat_history = []
# query = "What are 3 types of illegal hits that a jammer can easily make?"
# result = qa_chain({"question": query,
#                    "chat_history": chat_history})

# This attempt worked in the early version of Autoref
    # chat_history = []
    # while True:
    #     print("Your question:")
    #     question = input()
    #     result = qa_chain({"question": question, "chat_history": chat_history})
    #     chat_history.append((question, result["answer"]))
    #     print(f"AI: {result['answer']}")

    # chat_history = []
    # query = "What is the shape of the track?"
    # result = qa_chain({"question": query, "chat_history": chat_history})
    # print(f"Human: {query}")
    # print(f"AI: {result['answer']}")

    # chat_history = [(query, result["answer"])]
    # query = "What is the question I just asked you?"
    # result = qa_chain({"question": query, "chat_history": chat_history})
    # print(f"Human: {query}")
    # print(f"AI: {result['answer']}")

    # return 0

In [21]:
chat_history = []
while True:
    print("Your question:")
    question = input()
    result = qa_chain({"question": question,
                       "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print(f"AI: {result['answer']}")

Your quesiton:
AI: Hi Jake! The required equipment for playing roller derby includes quad roller skates, a helmet, mouthguard, elbow pads, wrist guards, and knee pads. It's important to make sure that all equipment meets safety standards and fits properly to ensure maximum protection during gameplay.
Your quesiton:
AI: I did not ask you for your name or provide you with a name when discussing the required equipment for playing roller derby.
Your quesiton:
AI: No, inline skates are not permitted for playing roller derby. Skaters must wear quad-style roller skates during play.
Your quesiton:
AI: Quad-style roller skates are required for playing roller derby. Inline skates are not permitted.
Your quesiton:


KeyboardInterrupt: 